In [1]:
from dotenv import load_dotenv
load_dotenv()

import os 
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [5]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint

messages=[AIMessage(content=f"please tell me how can i help",name="LLMModel")]
messages.append(HumanMessage(content=f"I want to learn coding",name="Sai"))
messages.append(AIMessage(content=f"Which programming language you want to learn",name="LLMModel"))
messages.append(HumanMessage(content=f"I want to learn Python Programming Language",name="Sai"))

for message in messages:
    message.pretty_print()

================================== Ai Message ==================================
Name: LLMModel

please tell me how can i help
================================ Human Message =================================
Name: Sai

I want to learn coding
================================== Ai Message ==================================
Name: LLMModel

Which programming language you want to learn
================================ Human Message =================================
Name: Sai

I want to learn Python Programming Language


In [8]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
result=llm.invoke(messages)

In [9]:
result.response_metadata

{'token_usage': {'completion_tokens': 628,
  'prompt_tokens': 57,
  'total_tokens': 685,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-2024-08-06',
 'system_fingerprint': 'fp_a288987b44',
 'id': 'chatcmpl-Bn4W3o4jGJVpLE6mJBMg7LEvwu7Cw',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}